<a href="https://colab.research.google.com/github/Rogerio-mack/VLF/blob/main/goes_hek_m25_b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Flare times on a GOES XRS plot

How to plot flare times as provided by the HEK on a GOES XRS plot.


In [ ]:
!pip install sunpy[all]

In [ ]:
import matplotlib.pyplot as plt

from sunpy.net import Fido
from sunpy.net import attrs as a
from sunpy.time import parse_time
from sunpy.timeseries import TimeSeries

Let's grab GOES XRS data for a particular time of interest and the HEK flare
data for this time from the NOAA Space Weather Prediction Center (SWPC).
Here we are searching for data from the GOES-15 satellite and for the 1-min average time-sampled data.



In [ ]:
tr = a.Time('2024-10-01 00:00', '2024-11-01 00:00')
results = Fido.search(tr, a.Instrument.xrs & a.goes.SatelliteNumber(16) & a.Resolution("avg1m") | a.hek.FL & (a.hek.FRM.Name == 'SWPC'))

Then download the XRS data and load it into a TimeSeries.



In [ ]:
files = Fido.fetch(results)
goes = TimeSeries(files)

Next let's retrieve `~sunpy.net.hek.HEKTable` from the Fido result
and then load the first row from HEK results into ``flares_hek``.



In [ ]:
hek_results = results['hek']
flares_hek = hek_results[0]

In [ ]:
len(hek_results), len(goes)

Lets plot everything together.



In [ ]:
fig, ax = plt.subplots()
goes[0].plot(axes=ax)
ax.axvline(parse_time(flares_hek['event_peaktime']).datetime)
ax.axvspan(
    parse_time(flares_hek['event_starttime']).datetime,
    parse_time(flares_hek['event_endtime']).datetime,
    alpha=0.2, label=flares_hek['fl_goescls']
)
ax.legend(loc=2)
ax.set_yscale('log')
ax.set_xlim(tr.start.to_datetime(), tr.end.to_datetime())

plt.show()

In [ ]:
import numpy as np

np.array([10**-exp for exp in range(3, 11)])

In [ ]:
from datetime import timedelta

flare_type = 'X'
fig, ax = plt.subplots(figsize=(15, 5))

ax.plot(goes[0].to_dataframe()['xrsa'], lw=1, color='r',label='xrsa',alpha=0.8)
ax.plot(goes[0].to_dataframe()['xrsb'], lw=1, color='b',label='xrsb',alpha=0.8)

for i in range(1,len(goes)):
  # goes[i].plot(axes=ax)
  ax.plot(goes[i].to_dataframe()['xrsa'], lw=1, color='r',alpha=0.8)
  ax.plot(goes[i].to_dataframe()['xrsb'], lw=1, color='b',alpha=0.8)

nr_flares = 1
for i in range(len(hek_results)):
  flares_hek = hek_results[i]
  if str(flares_hek['fl_goescls'])[0] == str(flare_type):
    ax.axvline(parse_time(flares_hek['event_peaktime']).datetime,linestyle=':',color='k')
    ax.axvspan(
        parse_time(flares_hek['event_starttime']).datetime,
        parse_time(flares_hek['event_endtime']).datetime,
        alpha=0.2,) # label=flares_hek['fl_goescls'])
    ax.text(parse_time(flares_hek['event_peaktime']).datetime + timedelta(hours=2),15**-(3+(nr_flares%3)/6),str(flares_hek['fl_goescls']),fontsize=8)
    nr_flares = nr_flares+1

for exp in range(3, 10):
  ax.axhline(10**-exp, lw=0.5, color='grey', linestyle='dashed')

ax.legend(loc=2)
ax.set_yscale('log')
ax.set_xlim(tr.start.to_datetime(), tr.end.to_datetime())

ax2 = ax.twinx()
ax2.set_yscale('log')
ax2.set_yticks(np.array([10**-(exp+0.5) for exp in range(3,8)]))
ax2.set_yticklabels(['X','M','C','B','A'],fontweight='bold')
ax2.set_ylim(ax.get_ylim())

ax.set_title(f"GOES-16 XRS - Flares {flare_type}\n {tr.start.to_datetime().strftime('%Y-%m-%d %Hh%Mm%Ss')} - {tr.end.to_datetime().strftime('%Y-%m-%d %Hh%Mm%Ss')}")

plt.show()

In [ ]:
tr.start.to_datetime().strftime('%Y-%m-%d-%H-%M'), tr.end.to_datetime().strftime('%Y-%m-%d-%H-%M')

In [ ]:
goes[0]

In [ ]:
goes[0].to_dataframe()['xrsa']

In [ ]:
dir(goes[0])

In [ ]:
for key in hek_results[0].keys():
  print(key, hek_results[0][key])

In [ ]:
str(hek_results[0]['fl_goescls'])

In [ ]:
str(hek_results[0]['fl_goescls'])[0] == str('C')